In [ ]:
!pip install transformers torch sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 897.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
'''
#minimal pair to test "even" (NPI) vs "still" (affirmative counterpart) <-- using Horn's conjecture as the null hypothesis
sen1 = "he didn't even come to the party"
sen2 = "he still came to the party"
'''

In [ ]:
'''

NPI	Non-NPI	Negation Context (+NPI)	Affirmative Context (–NPI)
"ever"	"always"	"He hasn’t ever lied."	"He has always lied."
"at all"	"a lot"	"She didn’t care at all."	"She cared a lot."
"yet"	"already"	"They haven’t left yet."	"They have already left."

'''

In [ ]:
sen1 = "she didn't eat any cookies"
sen2 = "she ate some cookies"

In [ ]:
#using sentence ensures NPI context is included
def get_word_embedding(sentence, target_word):
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    word_index = tokens.index(target_word)
    return embeddings[0, word_index]

# Get embeddings for NPI "even" vs. affirmative "still"
any_embedding = get_word_embedding(sen1, "any")
some_embedding = get_word_embedding(sen2, "some")

# Compare cosine similarity
cos_sim = torch.nn.CosineSimilarity(dim=0)(any_embedding, some_embedding)
print(f"Similarity between NPI 'any' and 'some': {cos_sim.item():.4f}")

Similarity between NPI 'any' and 'some': 0.6215


In [ ]:
NPI_component = any_embedding - some_embedding
print(NPI_component[:20]) #first 20 dim of NPI component

tensor([-0.2753,  0.2725,  0.1050, -0.2943,  1.0625, -0.0761, -0.4951, -0.1408,
         0.5801, -0.1171, -0.1553, -0.5525, -0.3594, -0.8080,  0.4505,  1.1009,
         0.2360, -0.2033, -1.3499,  0.5561])


In [ ]:
def get_phrase_embedding(sentence, target_phrase):
    """Get embedding for a word or phrase. For phrases, averages subword embeddings."""
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[0]  # Shape: [seq_len, 768]
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    # Handle multi-word phrases (e.g., "at all" → ["at", "all"])
    target_subwords = tokenizer.tokenize(target_phrase)
    target_ids = tokenizer.convert_tokens_to_ids(target_subwords)

    # Find the start position of the phrase in the tokenized sentence
    start_indices = [
        i for i, id_ in enumerate(inputs["input_ids"][0])
        if id_ == target_ids[0]
    ]

    # Check if the full phrase matches starting at each candidate index
    for start in start_indices:
        if inputs["input_ids"][0][start:start+len(target_ids)].tolist() == target_ids:
            phrase_embeddings = embeddings[start:start+len(target_ids)]
            return torch.mean(phrase_embeddings, dim=0)  # Average subword embeddings

    raise ValueError(f"Phrase '{target_phrase}' not found in sentence.")

In [ ]:
'''
#list of NPIs to test:
any_sen = "he didn't come to any party" #any
ever_sen = "he didn't ever come to the party" #ever
at_all_sen = "he didn't come to the party at all" #at all
yet_sen = "he didn't come to the party yet" #yet

any_embedding = get_word_embedding(any_sen, "any")
ever_embedding = get_word_embedding(ever_sen, "ever")
at_all_embedding = get_phrase_embedding(at_all_sen, "at all")
yet_embedding = get_word_embedding(yet_sen, "yet")

print(any_embedding[:20])
print(ever_embedding[:20])
print(at_all_embedding[:20])
print(yet_embedding[:20])

SyntaxError: incomplete input (<ipython-input-8-741d6b62192c>, line 1)

In [ ]:
ever_sen = "he didn't ever eat cookies"
at_all_sen = "he didn't eat cookies at all"
yet_sen = "he didn't eat cookies yet"

ever_embedding = get_word_embedding(ever_sen, "ever")
at_all_embedding = get_phrase_embedding(at_all_sen, "at all")
yet_embedding = get_word_embedding(yet_sen, "yet")

print(ever_embedding[:20])
print(at_all_embedding[:20])
print(yet_embedding[:20])

tensor([-3.3650e-01,  1.0133e-01, -2.3406e-01,  1.2182e-01,  9.1366e-02,
         8.2712e-01,  1.5662e-01,  2.5682e+00,  4.4793e-03, -5.9263e-01,
         4.4164e-01, -3.6775e-01, -2.4526e-03,  9.8878e-02, -7.6785e-02,
         3.1086e-01, -6.8875e-02,  3.6594e-01, -6.0830e-01,  9.7151e-01])
tensor([-0.8572,  0.1375, -0.1912,  0.3357,  0.2951,  0.1033,  0.2109,  1.8540,
        -0.8988, -0.1008,  0.3217, -0.3940, -0.4983, -0.1777, -0.4319,  0.8044,
         0.1635, -0.2392, -0.0491,  0.4000])
tensor([ 0.2443, -0.8522, -0.7787,  0.1178,  0.0509, -0.4798,  0.0067,  0.8972,
         0.2624,  0.0638,  0.2673, -0.2604,  0.2644, -0.0498, -1.1987, -0.4413,
         0.7029,  0.3663, -0.4799,  0.5641])


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch

def find_top_k_neighbors(query_vector, k=5, model=model, tokenizer=tokenizer):
    """
    Find the top-k most similar tokens to a query vector in BERT's embedding space.
    """
    # Get BERT's static word embeddings (vocab_size x 768)
    embeddings = model.embeddings.word_embeddings.weight.data

    # Convert query vector to numpy and reshape for sklearn
    query_np = query_vector.cpu().numpy().reshape(1, -1)
    embeddings_np = embeddings.cpu().numpy()

    # Compute cosine similarities
    similarities = cosine_similarity(query_np, embeddings_np)[0]

    # Get top-k indices
    top_k_indices = np.argpartition(similarities, -k)[-k:]
    top_k_indices = top_k_indices[np.argsort(similarities[top_k_indices])[::-1]]

    # Get tokens and scores
    top_k_tokens = tokenizer.convert_ids_to_tokens(top_k_indices)
    top_k_scores = similarities[top_k_indices]

    return list(zip(top_k_tokens, top_k_scores))

In [ ]:
#TEST VECTORS:
test2 = ever_embedding - NPI_component  #always, at some point
test3 = at_all_embedding - NPI_component  #somewhat, quite
test4 = yet_embedding - NPI_component  #already

tests = [test2, test3, test4]

for test in tests:
  neighbors = find_top_k_neighbors(test,k=5)
  for token, score in neighbors:
      print(f"{token}: {score:.3f}")
  print("")

never: 0.168
ever: 0.147
once: 0.143
always: 0.138
often: 0.127

us: 0.123
some: 0.115
times: 0.114
intervention: 0.111
g: 0.110

might: 0.118
connor: 0.116
why: 0.115
you: 0.113
##fia: 0.109

